<a href="https://colab.research.google.com/github/ssrakuen/skripsi-notebook/blob/main/Resnet_Models_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorboard
!pip install gputil

  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7394 sha256=85b2c36b7b79060c1e2c30670d94aa9c272db939a7ad0979f92d48305e0de1d3
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [ ]:
import os
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import psutil
import time
from torch.utils.tensorboard import SummaryWriter

# Function to load a model
def load_model(model_path, model_class):
    model = model_class(weights=None)
    model.fc = torch.nn.Linear(model.fc.in_features, 4)  # Adjust based on your number of classes
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to test a model
def test_model(model, test_loader, device):
    y_true = []
    y_pred = []
    start_time = time.time()
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
    end_time = time.time()

    # Metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')
    conf_matrix = confusion_matrix(y_true, y_pred)

    # Resource usage
    process = psutil.Process(os.getpid())
    cpu_usage = process.cpu_percent()
    gpu_usage = torch.cuda.memory_allocated(device) / 1024**2  # GPU memory usage in MB
    memory_usage = process.memory_info().rss / 1024**2  # Memory usage in MB

    # Time used in seconds
    time_used = end_time - start_time

    return {
        'confusion_matrix': conf_matrix,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'cpu_usage': cpu_usage,
        'gpu_usage': gpu_usage,
        'memory_usage': memory_usage,
        'time_used_seconds': time_used
    }

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = datasets.ImageFolder(root='/content/drive/MyDrive/Machine_Learning/Datasets/sleep_pose_dataset/A3/test', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define directory paths for saved models
models_dir = '/content/drive/MyDrive/Machine_Learning/modelsA3/'

# List all model files in the directory
model_files = [f for f in os.listdir(models_dir) if f.endswith('.pth')]

# Dictionary to map model filenames to their respective classes
model_classes = {
    'resnet18': models.resnet18,
    'resnet34': models.resnet34,
    'resnet50': models.resnet50,
    'resnet101': models.resnet101,
    'resnet152': models.resnet152
}

# TensorBoard writer
writer = SummaryWriter(log_dir='/content/drive/MyDrive/Machine_Learning/runs/A3')

# Dictionary to store test results
test_results = {}

# Iterate over each model file and test it
for model_file in model_files:
    model_name = model_file.split('_')[0]
    model_path = os.path.join(models_dir, model_file)

    print(f'Testing {model_name} model from {model_file}')

    # Load model
    model_class = model_classes[model_name]
    model = load_model(model_path, model_class)
    model.to(device)

    # Test model
    result = test_model(model, test_loader, device)
    test_results[model_file] = result  # Use model_file as key to distinguish between models

    # Log the results to TensorBoard
    writer.add_scalar(f'{model_name}/Accuracy', result['accuracy'], 0)
    writer.add_scalar(f'{model_name}/Precision', result['precision'], 0)
    writer.add_scalar(f'{model_name}/Recall', result['recall'], 0)
    writer.add_scalar(f'{model_name}/F1_Score', result['f1'], 0)
    writer.add_scalar(f'{model_name}/CPU_Usage', result['cpu_usage'], 0)
    writer.add_scalar(f'{model_name}/GPU_Usage', result['gpu_usage'], 0)
    writer.add_scalar(f'{model_name}/Memory_Usage', result['memory_usage'], 0)
    writer.add_scalar(f'{model_name}/Time_Used_Seconds', result['time_used_seconds'], 0)

# Save the results to a .npy file
np.save('/content/drive/MyDrive/Machine_Learning/test_stats/A3/test_results.npy', test_results)

# Display test results
for model_file, result in test_results.items():
    print(f'Model File: {model_file}')
    print(f'Confusion Matrix:\n{result["confusion_matrix"]}')
    print(f'Accuracy: {result["accuracy"]:.4f}')
    print(f'Precision: {result["precision"]:.4f}')
    print(f'Recall: {result["recall"]:.4f}')
    print(f'F1 Score: {result["f1"]:.4f}')
    print(f'CPU Usage (%): {result["cpu_usage"]:.2f}')
    print(f'GPU Usage (MB): {result["gpu_usage"]:.2f}')
    print(f'Memory Usage (MB): {result["memory_usage"]:.2f}')
    print(f'Time Used (s): {result["time_used_seconds"]:.2f}')
    print('\n')

# Close the TensorBoard writer
writer.close()


Testing resnet18 model from resnet18_lr0.001_batch16.pth
Testing resnet18 model from resnet18_lr0.001_batch32.pth
Testing resnet18 model from resnet18_lr0.001_batch64.pth
Testing resnet18 model from resnet18_lr0.0001_batch16.pth
Testing resnet18 model from resnet18_lr0.0001_batch32.pth
Testing resnet18 model from resnet18_lr0.0001_batch64.pth
Testing resnet34 model from resnet34_lr0.001_batch16.pth
Testing resnet34 model from resnet34_lr0.001_batch32.pth
Testing resnet34 model from resnet34_lr0.001_batch64.pth
Testing resnet34 model from resnet34_lr0.0001_batch16.pth
Testing resnet34 model from resnet34_lr0.0001_batch32.pth
Testing resnet34 model from resnet34_lr0.0001_batch64.pth
Testing resnet50 model from resnet50_lr0.001_batch16.pth
Testing resnet50 model from resnet50_lr0.001_batch32.pth
Testing resnet50 model from resnet50_lr0.001_batch64.pth
Testing resnet50 model from resnet50_lr0.0001_batch16.pth
Testing resnet50 model from resnet50_lr0.0001_batch32.pth
Testing resnet50 model 

In [ ]:
import numpy as np

# Path to the .npy file
npy_file_path = '/content/drive/MyDrive/Machine_Learning/test_stats/test_results.npy'

# Load the .npy file
test_results = np.load(npy_file_path, allow_pickle=True).item()

# Print the test results
for model_file, result in test_results.items():
    print(f'Model File: {model_file}')
    print(f'Confusion Matrix:\n{result["confusion_matrix"]}')
    print(f'Accuracy: {result["accuracy"]:.4f}')
    print(f'Precision: {result["precision"]:.4f}')
    print(f'Recall: {result["recall"]:.4f}')
    print(f'F1 Score: {result["f1"]:.4f}')
    print(f'CPU Usage (%): {result["cpu_usage"]:.2f}')
    print(f'GPU Usage (MB): {result["gpu_usage"]:.2f}')
    print(f'Memory Usage (MB): {result["memory_usage"]:.2f}')
    print(f'Time Used (s): {result["time_used_seconds"]:.2f}')
    print('\n')


Model File: resnet18_lr0.001_batch16.pth
Confusion Matrix:
[[184   0   0]
 [  0 188   0]
 [  0   0 194]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
CPU Usage (%): 0.00
GPU Usage (MB): 64.58
Memory Usage (MB): 1502.14
Time Used (s): 3.89


Model File: resnet18_lr0.001_batch32.pth
Confusion Matrix:
[[184   0   0]
 [  0 188   0]
 [  0   0 194]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
CPU Usage (%): 0.00
GPU Usage (MB): 63.45
Memory Usage (MB): 1502.14
Time Used (s): 3.35


Model File: resnet18_lr0.001_batch64.pth
Confusion Matrix:
[[184   0   0]
 [  0 188   0]
 [  0   0 194]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
CPU Usage (%): 0.00
GPU Usage (MB): 64.20
Memory Usage (MB): 1502.14
Time Used (s): 3.24


Model File: resnet18_lr0.0001_batch16.pth
Confusion Matrix:
[[184   0   0]
 [  0 188   0]
 [  0   0 194]]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
CPU Usage (%): 0.00
GPU Usage (MB): 64